## A tour of MLJ

### Models, machines, basic training and testing

Let's load data and define train and test rows:

In [4]:
using MLJ
using DataFrames, Statistics

Xraw = rand(300,3)
y = exp(Xraw[:,1] - Xraw[:,2] - 2Xraw[:,3] + 0.1*rand(300))
X = DataFrame(Xraw)

train, test = partition(eachindex(y), 0.70); # 70:30 split

A *model* is a container for hyperparameters:

In [3]:
knn_model=KNNRegressor(K=10)

KNNRegressor(target_type = Float64,
             K = 10,
             metric = MLJ.KNN.euclidean,
             kernel = MLJ.KNN.reciprocal,)

Wrapping the model in data creates a *machine* which will store training outcomes.

In [6]:
knn = machine(knn_model, X, y)

Machine{KNNRegressor{Float64}} @ 8…80


Training on the training rows and evaluating on the test rows:

In [7]:
fit!(knn, rows=train)
yhat = predict(knn, X[test,:])
rms(yhat, y[test])

┌ Info: Training Machine{KNNRegressor{Float64}} @ 8…80.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:97


0.07503960134878083

Or, we could have skipped the train/test definitions and evaluated one line:

In [8]:
evaluate!(knn, resampling=Holdout(fraction_train=0.7), measure=rms)

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.7 
│ shuffle=false 
│ measure=MLJ.rms 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/resampling.jl:90


0.07503960134878083

Our machine/model constructions and associateed fit/predict syntax anticipates a powerful extension for building networks of learners described later. Changing a hyperparameter and re-evaluating:

In [9]:
knn_model.K = 20
evaluate!(knn, resampling=Holdout(fraction_train=0.7))  # `default_measure(knn) == rms` so `measure` kwarg can be dropped

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.7 
│ shuffle=false 
│ measure=MLJ.rms 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/resampling.jl:90


0.0812870568583946

### Homogeneous ensembles

Here's a bagged ensemble model for 20 K-nearest neighbour regressors:

In [10]:
ensemble_model = EnsembleModel(atom=knn_model, n=20)

MLJ.DeterministicEnsembleModel(atom = KNNRegressor(target_type = Float64,
                                                   K = 20,
                                                   metric = MLJ.KNN.euclidean,
                                                   kernel = MLJ.KNN.reciprocal,),
                               weights = Float64[],
                               bagging_fraction = 0.8,
                               rng_seed = 0,
                               n = 20,
                               parallel = true,)

It can be trained and tested the same as any other model:

In [25]:
ensemble = machine(ensemble_model, X, y)
estimates = evaluate!(ensemble, resampling=CV())

┌ Info: Evaluating using cross-validation. 
│ nfolds=6. 
│ shuffle=false 
│ measure=MLJ.rms 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/resampling.jl:134
Cross-validating: 100%[=========================] Time: 0:00:00


6-element Array{Float64,1}:
 0.11544682575396704
 0.08767210494261213
 0.18563993762669195
 0.1118040789836054 
 0.11230712269221298
 0.0479029180925492 

In [27]:
mean(estimates)

0.11012883134860646

### Systematic tuning

Let's simultaneously tune the ensemble's `bagging_fraction` and the K-nearest neighbour hyperparameter `K`. Since one of our models is a field of the other, we have nested hyperparameters:

In [28]:
params(ensemble_model)

(atom = (target_type = Float64,
         K = 20,
         metric = MLJ.KNN.euclidean,
         kernel = MLJ.KNN.reciprocal,),
 weights = Float64[],
 bagging_fraction = 0.8,
 rng_seed = 0,
 n = 20,
 parallel = true,)

To define a tuning grid, we construct ranges for the two parameters and collate these ranges following the same pattern above (omitting parameters that don't change):

In [30]:
B_range = range(ensemble_model, :bagging_fraction, lower= 0.5, upper=1.0, scale = :linear)
K_range = range(knn_model, :K, lower=1, upper=100, scale=:log10)
nested_ranges = (atom = (K = K_range,), 
                 bagging_fraction = B_range)

(atom = (K = NumericRange @ 8…60,),
 bagging_fraction = NumericRange @ 9…03,)

Now we choose a tuning strategy:

In [31]:
tuning = Grid(resolution=12)

Grid(resolution = 12,
     parallel = true,)

And a resampling strategy:

In [32]:
resampling = Holdout(fraction_train=0.8)

Holdout(fraction_train = 0.8,
        shuffle = false,)

And define a new model which wraps the these strategies around our ensemble model:

In [33]:
tuned_ensemble_model = TunedModel(model=ensemble_model, 
    tuning=tuning, resampling=resampling, nested_ranges=nested_ranges)

MLJ.DeterministicTunedModel(model = MLJ.DeterministicEnsembleModel(atom = KNNRegressor(target_type = Float64,
                                                                                       K = 20,
                                                                                       metric = MLJ.KNN.euclidean,
                                                                                       kernel = MLJ.KNN.reciprocal,),
                                                                   weights = Float64[],
                                                                   bagging_fraction = 0.8,
                                                                   rng_seed = 0,
                                                                   n = 20,
                                                                   parallel = true,),
                            tuning = Grid(resolution = 12,
                                          parallel = true,),
                     

Fitting the corresponding machine tunes the underlying model (in this case an ensemble) and retrains on all supplied data:

In [34]:
tuned_ensemble = machine(tuned_ensemble_model, X[train,:], y[train])
fit!(tuned_ensemble);

┌ Warning: No measure specified. Using measure=MLJ.rms. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:86
┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…21.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:97
Iterating over an 132-point grid: 100%[=========================] Time: 0:00:06
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:142


For each fitted machine, one may inspect a user-friendly version of the fitted parameters (as opposed to the hyperparameters stored in its model). In the current case this is the best ensemble model (trained on all available data):

In [41]:
fitted_params(tuned_ensemble)

(best_model = DeterministicEnsembleModel @ 7…17,)

In [42]:
ans.best_model

MLJ.DeterministicEnsembleModel(atom = KNNRegressor(target_type = Float64,
                                                   K = 5,
                                                   metric = MLJ.KNN.euclidean,
                                                   kernel = MLJ.KNN.reciprocal,),
                               weights = Float64[],
                               bagging_fraction = 0.9545454545454546,
                               rng_seed = 0,
                               n = 20,
                               parallel = true,)

The `report` method gives more detail on the tuning process:

In [44]:
report(tuned_ensemble) 

(parameter_names = ["atom.K" "bagging_fraction"],
 parameter_scales = Symbol[:log10 :linear],
 parameter_values = Any[1 0.5; 2 0.5; … ; 66 1.0; 100 1.0],
 measurements = [0.110887, 0.0885497, 0.0907758, 0.0872331, 0.106281, 0.116563, 0.140795, 0.161544, 0.202221, 0.250571  …  0.11701, 0.100138, 0.0766859, 0.0791168, 0.0899609, 0.107305, 0.120879, 0.141716, 0.171783, 0.211828],
 best_measurement = 0.07629253700136358,)

Evaluating the tuned model:

In [45]:
yhat = predict(tuned_ensemble, X[test,:])
rms(yhat, y[test])

0.06728562254284808

Or, using all the data, get cross-validation estimates, with cv-tuning on each fold complement (nested resampling):

In [47]:
tuned_ensemble = machine(tuned_ensemble_model, X, y)
estimates = evaluate!(tuned_ensemble, resampling=CV(nfolds=4), verbosity=2)

┌ Info: Evaluating using cross-validation. 
│ nfolds=4. 
│ shuffle=false 
│ measure=MLJ.rms 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/resampling.jl:134
Cross-validating:  20%[=====>                   ]  ETA: 0:00:00┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…53.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:97
Iterating over an 132-point grid: 100%[=========================] Time: 0:00:06
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:142
Cross-validating:  40%[==========>              ]  ETA: 0:00:09┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…53.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:97
Iterating over an 132-point grid: 100%[=========================] Time: 0:00:06
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:142
Cross-validating:  60%[============

4-element Array{Float64,1}:
 0.0841372293833117  
 0.12378073058633587 
 0.09185865896480484 
 0.054174179086290165

In [48]:
mean(estimates)

0.08848769950518565

### Learning networks

MLJ has a flexible interface for building networks from multiple machine learning elements, whose complexity extend beyond linear "pipelines", and with a minimal of added abstraction.

In MLJ, a *learning network* is a graph whose nodes apply an operation, such as `predict` or `transform`, using a fixed machine (requiring training) - or which, alternatively, applies a regular (untrained) mathematical operation to its input(s). In practice, a learning network works with *fixed* sources for its training/evaluation data, but can be built and tested in stages. By contrast, an *exported learning network* is a learning network exported as a stand-alone, re-usable `Model` object, to which all the MLJ `Model`  meta-algorthims can be applied (ensembling, systematic tuning, etc). 

As we shall see, exporting a learning network as a reusable model, is very easy. 

### Building a simple learning network

![](wrapped_ridge.png)

The diagram above depicts a learning network which standardises the input data, `X`, learns an optimal Box-Cox transformation for the target, `y`, predicts new targets using ridge regression, and then inverse-transforms those predictions (for later comparison with the original test data). The machines are labelled yellow. 

To implement the network, we begin by loading all data needed for training and evaluation into *source nodes*:

In [49]:
Xs = source(X)
ys = source(y)

Source @ 1…72

We label nodes according to their outputs in the diagram. Notice that the nodes `z` and `yhat` use the same machine `box` for different operations. 

To construct the `W` node we first need to define the machine `stand` that it will use to transform inputs. 

In [50]:
stand_model = Standardizer()
stand = machine(stand_model, Xs)

NodalMachine @ 1…47 = machine(Standardizer @ 7…49, 6…88)

Because `Xs` is a node, instead of concrete data, we can call `transform` on the machine without first training it, and the result is the new node `W`, instead of concrete transformed data:

In [51]:
W = transform(stand, Xs)

Node @ 7…87 = transform(1…47, 6…88)

To get actual transformed data we *call* the node appropriately, which will require we first train the node. Training a node, rather than a machine, triggers training of *all* necessary machines in the network.

In [52]:
fit!(W, rows=train)
W()          # transform all data
W(rows=test) # transform only test data
W(X[3:4,:])  # transform any data, new or old

┌ Info: Training NodalMachine{Standardizer} @ 1…47.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:97


(x1 = [-0.38189, -0.875234],
 x2 = [-0.906512, -1.11498],
 x3 = [0.500238, -0.1854],)

If you like, you can think of `W` (and the other nodes we will define) as "dynamic data": `W` is *data*, in the sense that  it an be called ("indexed") on rows, but *dynamic*, in the sense the result depends on the outcome of training events. 

The other nodes of our network are defined similarly:

In [53]:
box_model = UnivariateBoxCoxTransformer()  # for making data look normally-distributed
box = machine(box_model, ys)
z = transform(box, ys)

ridge_model = RidgeRegressor(lambda=0.1)
ridge =machine(ridge_model, W, z)
zhat = predict(ridge, W)

yhat = inverse_transform(box, zhat)

Node @ 1…99 = inverse_transform(1…12, predict(8…85, transform(1…47, 6…88)))

We are ready to train and evaluate the completed network. Notice that the standardizer, `stand`, is *not* retrained, as MLJ remembers that it was trained earlier:

In [54]:
fit!(yhat, rows=train)
rms(y[test], yhat(rows=test)) # evaluate

┌ Info: Not retraining NodalMachine{Standardizer} @ 1…47. It is up-to-date.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/networks.jl:201
┌ Info: Training NodalMachine{UnivariateBoxCoxTransfor…} @ 1…12.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:97
┌ Info: Training NodalMachine{RidgeRegressor{Float64}} @ 8…85.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:97


0.036221810995086694

In [55]:
yhat(X[3:4,:])  # predict on new or old data

2-element Array{Float64,1}:
 0.33839918522447765
 0.46302135836541963

We can change hyperparameters and retrain:

In [56]:
ridge_model.lambda = 0.01
fit!(yhat, rows=train) 
rms(y[test], yhat(rows=test))

┌ Info: Not retraining NodalMachine{UnivariateBoxCoxTransfor…} @ 1…12. It is up-to-date.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/networks.jl:201
┌ Info: Not retraining NodalMachine{Standardizer} @ 1…47. It is up-to-date.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/networks.jl:201
┌ Info: Updating NodalMachine{RidgeRegressor{Float64}} @ 8…85.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:101


0.03645204707395768

> **Notable feature.** The machine, `ridge::NodalMachine{RidgeRegressor}`, is retrained, because its underlying model has been mutated. However, since the outcome of this training has no effect on the training inputs of the machines `stand` and `box`, these transformers are left untouched. (During construction, each node and machine in a learning network determines and records all machines on which it depends.) This behaviour, which extends to exported learning networks, means we can tune our wrapped regressor without re-computing transformations each time the hyperparameter is changed. 

### Exporting a learning network as a composite model

To export a learning network:
- Define a new `mutable struct` model type.
- Wrap the learning network code in a model `fit` method.

All learning networks that make determinisic (or, probabilistic) predictions export as models of subtype `Deterministic{Node}` (respectively, `Probabilistic{Node}`):


In [57]:
mutable struct WrappedRidge <: Deterministic{Node}
    ridge_model
end

Now satisfied that our wrapped Ridge Regression learning network works, we simply cut and paste its defining code into a `fit` method: 

In [58]:
function MLJ.fit(model::WrappedRidge, X, y)
    Xs = source(X)
    ys = source(y)

    stand_model = Standardizer()
    stand = machine(stand_model, Xs)
    W = transform(stand, Xs)

    box_model = UnivariateBoxCoxTransformer()  # for making data look normally-distributed
    box = machine(box_model, ys)
    z = transform(box, ys)

    ridge_model = model.ridge_model ###
    ridge =machine(ridge_model, W, z)
    zhat = predict(ridge, W)

    yhat = inverse_transform(box, zhat)
    fit!(yhat, verbosity=0)
    
    return yhat
end

The line marked `###`, where the new exported model's hyperparameter `ridge_model` is spliced into the network, is the only modification.

This completes the export process.

> **What's going on here?** MLJ's machine interface is built atop a more primitive *[model](adding_new_models.md)* interface, implemented for each algorithm. Each supervised model type (eg, `RidgeRegressor`) requires model `fit` and `predict` methods, which are called by the corresponding machine `fit!` and `predict` methods. We don't need to define a  model `predict` method here because MLJ provides a fallback which simply calls the node returned by `fit` on the data supplied: `MLJ.predict(model::Supervised{Node}, Xnew) = yhat(Xnew)`.

Let's now let's wrap our composite model as a tuned model and evaluate on the Boston dataset:

In [59]:
task = load_boston()
X, y = task()
train, test = partition(eachindex(y), 0.7)
wrapped_model = WrappedRidge(ridge_model)

WrappedRidge(ridge_model = RidgeRegressor(target_type = Float64,
                                          lambda = 0.01,),)

In [60]:
params(wrapped_model)

(ridge_model = (target_type = Float64,
                lambda = 0.01,),)

In [61]:
nested_ranges = (ridge_model = (lambda = range(ridge_model, :lambda, lower=0.1, upper=100.0, scale=:log10),),)

(ridge_model = (lambda = NumericRange @ 1…58,),)

In [62]:
tuned_wrapped_model = TunedModel(model=wrapped_model, tuning=Grid(resolution=20),
resampling=CV(), measure=rms, nested_ranges=nested_ranges);

In [63]:
tuned_wrapped = machine(tuned_wrapped_model, X, y)
evaluate!(tuned_wrapped, resampling=Holdout(fraction_train=0.7), measure=rms, verbosity=0) |> mean  # nested resampling estimate

6.88236977264247